<a href="https://colab.research.google.com/github/kunai-3txk/Compe_tonyobyo/blob/main/AMP_ProPepEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns; sns.set()

#config

In [3]:
class paths:
    # kaggle環境ならTrue
    if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
        common_path = "/kaggle/input/amp-parkinsons-disease-progression-prediction"
    
    # colaboratory環境ならTrue
    if 'COLAB_GPU' in set(os.environ.keys()):
        common_path = "/content/drive/MyDrive/AMP"
    
    SAMPLE_SUBMISSION = common_path + "/example_test_files/sample_submission.csv"
    SUPPLEMENTAL_CLINICAL_DATA = common_path + "/supplemental_clinical_data.csv"
    TRAIN_CLINICAL_DATA = common_path + "/train_clinical_data.csv"
    TRAIN_PEPTIDES = common_path + "/train_peptides.csv"
    TRAIN_PROTEINS = common_path + "/train_proteins.csv"
    TEST_CLINICAL_DATA = common_path + "/example_test_files/test.csv"
    TEST_PEPTIDES = common_path + "/example_test_files/test_peptides.csv"
    TEST_PROTEINS = common_path + "/example_test_files/test_proteins.csv"

#load data

In [4]:
train_clinical = pd.read_csv(paths.TRAIN_CLINICAL_DATA)
train_peptides = pd.read_csv(paths.TRAIN_PEPTIDES)
train_proteins = pd.read_csv(paths.TRAIN_PROTEINS)
supplemental_clinical = pd.read_csv(paths.SUPPLEMENTAL_CLINICAL_DATA)
#print('Correct label:Proteins shape: (232741, 5) Peptides shape: (981834, 6) Clinical shape: (2615, 8)')
print('Proteins shape:',train_proteins.shape)
print('Peptides shape:',train_peptides.shape)
print('Clinical shape:',train_clinical.shape)
print('supplemental_clinical shape:',supplemental_clinical.shape)

Proteins shape: (232741, 5)
Peptides shape: (981834, 6)
Clinical shape: (2615, 8)
supplemental_clinical shape: (2223, 8)


In [5]:
test_clinical = pd.read_csv(paths.TEST_CLINICAL_DATA)
test_peptides = pd.read_csv(paths.TEST_PEPTIDES)
test_proteins = pd.read_csv(paths.TEST_PROTEINS)
sample_submission= pd.read_csv(paths.SAMPLE_SUBMISSION)
print('Proteins shape:',test_proteins.shape)
print('Peptides shape:',test_peptides.shape)
print('Clinical shape:',test_clinical.shape)

Proteins shape: (453, 6)
Peptides shape: (2057, 7)
Clinical shape: (16, 6)


#EDA

In [ ]:
proteins = []
protein_dict = {}
for index, row in train_proteins.iterrows():
    protein = row["UniProt"]
    if protein not in protein_dict:
        protein_dict[protein] = {}
        proteins.append(protein)
    protein_dict[protein][row["visit_id"]] = row["NPX"]
    
peptides = []
peptide_dict = {}
for index, row in train_peptides.iterrows():
    peptide = row["Peptide"]
    if peptide not in peptide_dict:
        peptide_dict[peptide] = {}
        peptides.append(peptide)
    peptide_dict[peptide][row["visit_id"]] = row["PeptideAbundance"]
    
train_copy = train_clinical.copy()
for protein in proteins:
    train_copy[protein] = train_copy["visit_id"].apply(lambda visit_id: 0 if visit_id not in protein_dict[protein] else protein_dict[protein][visit_id])
    
for peptide in peptides:
    train_copy[peptide] = train_copy["visit_id"].apply(lambda visit_id: 0 if visit_id not in peptide_dict[peptide] else peptide_dict[peptide][visit_id])

In [31]:
features_pro = []
features_pro.extend(proteins)

train_copy[features_pro] = train_copy[features_pro].replace(0.0, np.nan)
features_pro.extend(["updrs_1", "updrs_2", "updrs_3", "updrs_4"])
correlation_matrix_pro = train_copy[features_pro].corr(method="spearman")

,GAYPLSIEPIGVR,EPGLC(UniMod_4)TWQSLR,WQEEMELYR,QPSSAFAAFVK,GLVSWGNIPC(UniMod_4)GSK,updrs_1,updrs_2,updrs_3,updrs_4
GAYPLSIEPIGVR,1.000000,0.250925,0.026724,0.568750,0.138107,0.003868,-0.027103,-0.029396,-0.031724
EPGLC(UniMod_4)TWQSLR,0.250925,1.000000,-0.228678,0.328199,0.620028,-0.021548,-0.032427,-0.012249,-0.037525
WQEEMELYR,0.026724,-0.228678,1.000000,-0.053570,-0.305838,0.009482,0.038948,0.032230,-0.011492
QPSSAFAAFVK,0.568750,0.328199,-0.053570,1.000000,0.295019,-0.065151,-0.058640,-0.033580,-0.027887
GLVSWGNIPC(UniMod_4)GSK,0.138107,0.620028,-0.305838,0.295019,1.000000,-0.098834,-0.080189,-0.076168,0.025481
updrs_1,0.003868,-0.021548,0.009482,-0.065151,-0.098834,1.000000,0.629798,0.433025,0.382011
updrs_2,-0.027103,-0.032427,0.038948,-0.058640,-0.080189,0.629798,1.000000,0.769814,0.356639
updrs_3,-0.029396,-0.012249,0.032230,-0.033580,-0.076168,0.433025,0.769814,1.000000,0.211685
updrs_4,-0.031724,-0.037525,-0.011492,-0.027887,0.025481,0.382011,0.356639,0.211685,1.000000


In [34]:
features_pep = []
features_pep.extend(peptides)

train_copy[features_pep] = train_copy[features_pep].replace(0.0, np.nan)
features_pep.extend(["updrs_1", "updrs_2", "updrs_3", "updrs_4"])
correlation_matrix_pep = train_copy[features_pep].corr(method="spearman")

,NEQEQPLGQWHLS,GNPEPTFSWTK,IEIPSSVQQVPTIIK,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,SMEQNGPGLEYR,TLKIENVSYQDKGNYR,VIAVNEVGR,VMTPAVYAPYDVK,VNGSPVDNHPFAGDVVFPR,ELDLNSVLLK,...,PALEDLR,QKVEPLRAELQEGAR,RTHLPEVFLSK,INHC(UniMod_4)RFDEFFSEGC(UniMod_4)APGSKK,EQPPSLTR,AGC(UniMod_4)VAESTAVC(UniMod_4)R,updrs_1,updrs_2,updrs_3,updrs_4
NEQEQPLGQWHLS,1.000000,0.453816,0.513447,0.499844,0.410791,0.428599,0.415754,0.442860,0.514718,0.326326,...,-0.180604,-0.277398,-0.040064,0.215683,-0.038296,0.134909,-0.049112,-0.104992,-0.083040,-0.005270
GNPEPTFSWTK,0.453816,1.000000,0.729279,0.810803,0.878452,0.819320,0.843704,0.834792,0.800927,0.472696,...,-0.144591,-0.211338,-0.164159,0.390371,0.053320,0.162099,-0.142315,-0.160321,-0.178667,-0.018186
IEIPSSVQQVPTIIK,0.513447,0.729279,1.000000,0.867775,0.734973,0.741122,0.743443,0.659319,0.811577,0.520504,...,-0.188206,-0.270042,-0.212821,0.317972,-0.042276,0.093827,-0.121384,-0.175655,-0.219944,-0.025602
KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,0.499844,0.810803,0.867775,1.000000,0.819926,0.782280,0.811231,0.720601,0.852049,0.473910,...,-0.126635,-0.234017,-0.235237,0.429488,0.038166,0.095195,-0.106764,-0.167879,-0.221889,0.033778
SMEQNGPGLEYR,0.410791,0.878452,0.734973,0.819926,1.000000,0.867684,0.909782,0.828873,0.844779,0.508401,...,-0.108375,-0.176271,-0.125145,0.405253,0.110960,0.156838,-0.111435,-0.144350,-0.153001,-0.016753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AGC(UniMod_4)VAESTAVC(UniMod_4)R,0.134909,0.162099,0.093827,0.095195,0.156838,0.183879,0.199392,0.216150,0.124870,0.206241,...,0.059254,-0.022982,0.060824,0.079225,0.233738,1.000000,0.060005,0.114605,0.097508,-0.034325
updrs_1,-0.049112,-0.142315,-0.121384,-0.106764,-0.111435,-0.112040,-0.111249,-0.141670,-0.084752,-0.041213,...,0.041818,0.031451,0.046925,-0.080959,-0.001511,0.060005,1.000000,0.629798,0.433025,0.382011
updrs_2,-0.104992,-0.160321,-0.175655,-0.167879,-0.144350,-0.181169,-0.163496,-0.157840,-0.157359,-0.016560,...,-0.007957,0.040672,-0.018188,-0.116742,-0.051948,0.114605,0.629798,1.000000,0.769814,0.356639
updrs_3,-0.083040,-0.178667,-0.219944,-0.221889,-0.153001,-0.213493,-0.189544,-0.161090,-0.194561,0.003090,...,-0.016159,0.025222,-0.021495,-0.141099,-0.049286,0.097508,0.433025,0.769814,1.000000,0.211685


In [38]:
updrs1_pro_list = []
updrs2_pro_list = []
updrs3_pro_list = []
updrs4_pro_list = []
threshold = 0.12

for index, row in correlation_matrix_pro[['updrs_1','updrs_2','updrs_3','updrs_4']].iterrows():
  if abs(row['updrs_1']) >= threshold :
    updrs1_pro_list.append([index,row['updrs_1']])
  if abs(row['updrs_2']) >= threshold:
    updrs2_pro_list.append([index,row['updrs_2']])
  if abs(row['updrs_3']) >= threshold:
    updrs3_pro_list.append([index,row['updrs_3']])
  if abs(row['updrs_4']) >= threshold:
    updrs4_pro_list.append([index,row['updrs_4']])



NameError: ignored

In [43]:
updrs1_pep_list = []
updrs2_pep_list = []
updrs3_pep_list = []
updrs4_pep_list = []
threshold = 0.12
for index, row in correlation_matrix_pep[['updrs_1','updrs_2','updrs_3','updrs_4']].iterrows():
  if abs(row['updrs_1']) >= threshold :
    updrs1_pep_list.append([index,row['updrs_1']])
  if abs(row['updrs_2']) >= threshold:
    updrs2_pep_list.append([index,row['updrs_2']])
  if abs(row['updrs_3']) >= threshold:
    updrs3_pep_list.append([index,row['updrs_3']])
  if abs(row['updrs_4']) >= threshold:
    updrs4_pep_list.append([index,row['updrs_4']])

[['GNPEPTFSWTK', -0.14231510019992835],
 ['IEIPSSVQQVPTIIK', -0.12138427563422904],
 ['VMTPAVYAPYDVK', -0.1416701139329099],
 ['AYQGVAAPFPK', -0.13757036157227495],
 ['QQETAAAETETR', -0.14158660453842506],
 ['NIINSDGGPYVC(UniMod_4)R', -0.12712811010786054],
 ['QALNTDYLDSDYQR', -0.13342189780237623],
 ['AIGYLNTGYQR', -0.12717498631438212],
 ['ALDFAVGEYNK', -0.1259164414484765],
 ['LVGGPMDASVEEEGVR', -0.12494728401205199],
 ['AATVGSLAGQPLQER', -0.16577001070086375],
 ['AKLEEQAQQIR', -0.14489267700862057],
 ['AYKSELEEQLTPVAEETR', -0.13223499677550318],
 ['DRLDEVKEQVAEVR', -0.14022951781277454],
 ['GEVQAMLGQSTEELR', -0.13014946164009988],
 ['GEVQAMLGQSTEELRVR', -0.12829171877310727],
 ['LDEVKEQVAEVR', -0.1237100752227691],
 ['LEEQAQQIR', -0.12709021522656008],
 ['LGPLVEQGR', -0.1311524212574657],
 ['LGPLVEQGRVR', -0.15049547688867893],
 ['LQAEAFQAR', -0.15490856151578253],
 ['LSKELQAAQAR', -0.1402160884483458],
 ['QWAGLVEK', -0.16940641136558332],
 ['SELEEQLTPVAEETR', -0.1279544003375494],